# StudyBuddy Prototyping
copied from Introduction to LangChain v0.1.0 and LCEL: LangChain Powered RAG


#### TaInstalling Required Libraries


In [1]:
from qdrant_client import QdrantClient
client = QdrantClient(path="persist_dir") 
help(client)

Help on QdrantClient in module qdrant_client.qdrant_client object:

class QdrantClient(qdrant_client.qdrant_fastembed.QdrantFastembedMixin)
 |  QdrantClient(location: Optional[str] = None, url: Optional[str] = None, port: Optional[int] = 6333, grpc_port: int = 6334, prefer_grpc: bool = False, https: Optional[bool] = None, api_key: Optional[str] = None, prefix: Optional[str] = None, timeout: Optional[int] = None, host: Optional[str] = None, path: Optional[str] = None, force_disable_check_same_thread: bool = False, grpc_options: Optional[Dict[str, Any]] = None, auth_token_provider: Union[Callable[[], str], Callable[[], Awaitable[str]], NoneType] = None, **kwargs: Any)
 |  
 |  Entry point to communicate with Qdrant service via REST or gRPC API.
 |  
 |  It combines interface classes and endpoint implementation.
 |  Additionally, it provides custom implementations for frequently used methods like initial collection upload.
 |  
 |  All methods in QdrantClient accept both gRPC and REST str

In [ ]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

Now we can get our Qdrant dependencies!

In [ ]:
!pip install -qU qdrant-client

Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [ ]:
!pip install -qU tiktoken pymupdf

#### Set Environment 

We will read from .env
and use config to control the constants

In [1]:
path = 'studybuddy_utils/'

In [2]:
from dotenv import load_dotenv
load_dotenv()

# tbd make sure, the key has been loaded

True

In [1]:
from studybuddy_utils.config import Config

Set up LangSmith

In [2]:
from uuid import uuid4
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"StudyBuddy - Development - {uuid4().hex[0:8]}"


#### Read and chunk docs

In [3]:
from studybuddy_utils.text_utils import SBDocLoader
sbdocs_loader = SBDocLoader(path+Config.pdf_path)

chunks = sbdocs_loader.load_and_parse_pdf_dir()

NameError: name 'path' is not defined

### Create embeddings and setup vector store

In [7]:
from studybuddy_utils.vectorstore import IndexBuilder
index = IndexBuilder(chunks)
qdrant_retriever = index.retriever

EmbeddingModel set text-embedding-ada-002


### Formulate the question

In [9]:
question = "What is the meaning of the number 42 in Douglas Adam's Hitchhicker's Guide?"

### Execute the chain

In [10]:
from studybuddy_utils.reasoning import SimpleChain
chain = SimpleChain(qdrant_retriever)
response = chain.reason(question)

ChatModel set gpt-3.5-turbo


### Show the result

In [11]:
response

"The meaning of the number 42 in Douglas Adam's Hitchhicker's Guide is not explicitly mentioned in the provided context. Therefore, the answer is: 'I don't know the answer to that question based on the provided context.'."

# Experiment with a big dataset

In [4]:
from studybuddy_utils.config import Config
path = 'studybuddy_utils/'
pdf_path = 'data/AI-Powered_Search_v20.pdf'
pdf_path ='data/sudel.pdf'
pdf_path = 'data/AI-Powered_Search_v20.pdf'
session_uuid='dev'

In [5]:
from studybuddy_utils.text_utils import SBDocLoader
sbdocs_loader = SBDocLoader('')
chunks = sbdocs_loader.load_and_parse_pdf(pdf_path)


# of chunks: 1523
# max chunk len: 199


In [6]:
async def load_and_parse(path: str):
    documents =  await parser.aload_data(path)
    return documents

Try LlamaIndex instead

In [7]:
import nest_asyncio

nest_asyncio.apply()

from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True,
)

# sync
# documents = parser.load_data("./my_file.pdf")

# sync batch
# documents = parser.load_data(["./my_file1.pdf", "./my_file2.pdf"])

# async
documents = load_and_parse(pdf_path)

# async batch
# documents = await parser.aload_data(["./my_file1.pdf", "./my_file2.pdf"])#

In [8]:
documents

<coroutine object load_and_parse at 0x174139700>

# Experiment the fine-tuned model

In [ ]:
!pip install sentence_transformers


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

from huggingface_hub import login

login(token=os.getenv('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/ursuladeriu/.cache/huggingface/token
Login successful


In [4]:
from studybuddy_utils.config import Config
from langchain_community.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader(Config.pdf_path).load()

# Evaluator


In [20]:
# from studybuddy_utils.prompts import EvaluationPrompt
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

from langchain_core.prompts import ChatPromptTemplate

In [21]:
system_prompt = """
You are a university professor grading a quiz in information retrieval.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?


The question is given below:

---------------------
{question}
---------------------

Given the question score the user's answer based on the ideal answer provided here:

---------------------
{ideal_answer}
---------------------
Please take your time, and think through each item step-by-step.
If you don't know then simply provide scores -1.
When you are done -  please provide your response in a JSON format as follows:

"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"
"""




In [23]:
human_prompt = """
Based on the information given, score the following answer:
---------------------
{student_answer}
---------------------

"""

In [19]:
aa=system_prompt.format(question='gugus', ideal_answer='dada')
# print(aa)
bb=human_prompt.format(student_answer='juhu')
#print(bb)


Based on the information given, score the following answer:
---------------------
juhu
---------------------




In [35]:
question = "Define the term: precision"
ideal_answer = "Precision is the fraction of relevant instances among the retrieved instances."
answer = "It is simply not the same as recall"

In [37]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt)
])
chat_prompt

ChatPromptTemplate(input_variables=['ideal_answer', 'question', 'student_answer'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ideal_answer', 'question'], template='\nYou are a university professor grading a quiz in information retrieval.\n\nYou should be hyper-critical.\n\nProvide scores (out of 10) for the following attributes:\n\n1. Clarity - how clear is the response\n2. Faithfulness - how related to the original query is the response\n3. Correctness - was the response correct?\n\n\nThe question is given below:\n\n---------------------\n{question}\n---------------------\n\nGiven the question score the user\'s answer based on the ideal answer provided here:\n\n---------------------\n{ideal_answer}\n---------------------\nPlease take your time, and think through each item step-by-step.\nIf you don\'t know then simply provide scores -1.\nWhen you are done -  please provide your response in a JSON format as follows:\n\n"clarity" : "score_out_of_10", "f

In [38]:
chain = chat_prompt | openai_chat_model

In [39]:
response = chain.invoke({"question": question, 
                        "ideal_answer": ideal_answer,
                        "student_answer": answer})

In [41]:
response.content

'"clarity" : "2", "faithfulness" : "1", "correctness" : "0"'